# Практическое задание к курсу

**1. Обучите нейронную сеть любой архитектуры которой не было на курсе, либо обучите нейронную сеть разобранной архитектуры, но на том датасете, которого не было на уроках. Сделайте анализ, того, что вам помогло в улучшения работы нейронной сети.**

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import copy, numpy as np
np.random.seed(0)
from __future__ import print_function
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Activation
from keras.layers import LSTM
from keras.datasets import imdb
from keras.layers.recurrent import SimpleRNN, LSTM, GRU

Using TensorFlow backend.


Исходный вариант из методички

In [3]:
# построчное чтение из примера с текстом 
with open("/content/drive/My Drive/Colab Notebooks/Deep mind's Go.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)


# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])


# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1


# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 128, 128
NUM_ITERATIONS = 1 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100


# Create a super simple recurrent neural network. There is one recurrent
# layer that produces an embedding of size HIDDEN_SIZE from the one-hot
# encoded input layer. This is followed by a Dense fully-connected layer
# across the set of possible next characters, which is converted to a
# probability score via a standard softmax activation with a multi-class
# cross-entropy loss function linking the prediction to the one-hot
# encoding character label.

'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. За ним последует полносвязный слой связанный с набором возможных следующих символов, которые конвертированы в вероятностные результаты через стандартную softmax активацию с multi-class cross-encoding loss функцию ссылающуются на предсказание one-hot encoding лейбл символа
'''

model = Sequential()
model.add(
    GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")


# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()


Итерация #: 0
Epoch 1/1
73895/73895 [==============================] - 22s 300us/step - loss: 2.7884
Генерация из посева: 15. all of
15. all of antion antion antion antion antion antion antion antion antion antion antion antion antion antion a


Увеличим NUM_ITERATIONS с 1 до 25

In [6]:
# построчное чтение из примера с текстом 
with open("/content/drive/My Drive/Colab Notebooks/Deep mind's Go.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)


# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])


# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1


# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 128, 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100


# Create a super simple recurrent neural network. There is one recurrent
# layer that produces an embedding of size HIDDEN_SIZE from the one-hot
# encoded input layer. This is followed by a Dense fully-connected layer
# across the set of possible next characters, which is converted to a
# probability score via a standard softmax activation with a multi-class
# cross-entropy loss function linking the prediction to the one-hot
# encoding character label.

'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. За ним последует полносвязный слой связанный с набором возможных следующих символов, которые конвертированы в вероятностные результаты через стандартную softmax активацию с multi-class cross-encoding loss функцию ссылающуются на предсказание one-hot encoding лейбл символа
'''

model = Sequential()
model.add(
    GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")


# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()


Итерация #: 0
Epoch 1/1
73895/73895 [==============================] - 22s 298us/step - loss: 2.7767
Генерация из посева: th higher 
th higher the the the the the the the the the the the the the the the the the the the the the the the the the ==================================================
Итерация #: 1
Epoch 1/1
73895/73895 [==============================] - 21s 289us/step - loss: 2.2628
Генерация из посева: ee extende
ee extended the search to the search to the search to the search to the search to the search to the search to ==================================================
Итерация #: 2
Epoch 1/1
73895/73895 [==============================] - 21s 289us/step - loss: 2.0408
Генерация из посева: algorithms
algorithms the search programs and the search programs and the search programs and the search programs and the==================================================
Итерация #: 3
Epoch 1/1
73895/73895 [==============================] - 21s 290us/step - loss: 1.8885
Генерация из посе

**Выводы:** 

1. сгенерирован текст planes dethors were allont games, r. & milery search the extended data table 1 | detains in the game of go. 

2. loss существенно уменьшился по сравнению с первой (одной) итерацией. Текст сгенерирован более или менее осмысленный и по теме.

**2. Сделайте краткий обзор какой-нибудь научной работы посвященной тому или иному алгоритму нейронных сетей, который не рассматривался на курсе. Проведите анализ: Чем отличается выбранная вами на рассмотрение архитектура нейронной сети от других архитектур? В чем плюсы и минусы данной архитектуры? Какие могут возникнуть трудности при применении данной архитектуры на практике?**

Адрес научной статьи: https://storage.googleapis.com/deepmind-media/alphago/AlphaGoNaturePaper.pdf

AlphaGo - это программно-аппаратный комплекс для игры в Го.

Го - это древняя игра. Ее родиной считается Китай, либо Япония (оспаривается).

AlphaGo - разработана Google DeepMind в 2015 году. AlphaGo была первой в мире программой, которая выиграла профессионального Го-игрока без гандикапа (без форы) на стандартной доске 19 × 19 клеток.

Эта победа стала важным прорывом в области искусственного интеллекта, поскольку большинство экспертов по искусственному интеллекту полагали, что такая программа не будет создана до 2020–2025 гг.
Все предыдущие попытки создать программу аналогичной силы с использованием классического программирования заканчивались неудачей.

Существенной особенностью AlphaGo является то, что она использует общие алгоритмы, которые практически не зависят от особенностей игры. Алгоритм AlphaGo содержит только основные принципы игры, с которых любой начинающий игрок начинает изучать игру. Например, подсчет количества свободных очков из группы камней и анализ способности захватывать камни с помощью «лестницы». Остальное AlphaGo изучил самостоятельно, проанализировав базу данных из 160 тысяч игр, используя общие методы, которые можно использовать в других областях искусственного интеллекта.

AlphaGo работает с использованием нейронных сетей, которые успешно используются в распознавании образов. Крупный прорыв в этой области был связан с использованием многоуровневых сверточных нейронных сетей и специальной методики их обучения - глубокого обучения. Сверточные нейронные сети состоят из нескольких уровней нейронов. Каждый уровень получает матрицу чисел на входе, объединяет их с некоторыми весами и, используя нелинейную функцию активации, выдает много чисел на выходе, которые передаются на следующий уровень. При распознавании изображений изображение подается на первый уровень, а последний уровень дает результат. Нейронные сети тренируются на большом количестве изображений, постоянно корректируя веса, используемые для расчета результата. В результате нейронная сеть имеет конфигурацию, которая способна распознавать похожие изображения. Этот процесс не может быть предсказан, поэтому трудно сказать, как «мыслит» нейронная сеть, но, грубо говоря, выходные данные на промежуточных уровнях соответствуют различным возможным классификациям.

Представление игровой позиции

AlphaGo использует свёрточные нейронные сети для того, чтобы оценить позицию или предсказать следующий ход. Аналогично тому, как при распознавании образов на нейронную сеть подаётся изображение, AlphaGo подаёт на нейронные сети позицию. Каждая позиция представлена как многослойная картинка 19x19, где каждый слой представляет описания простых свойств каждого пункта доски. Используются следующие простые свойства: цвет камня, количество свободных пунктов у данной группы камней (если их не больше 8), взятие камней, возможность сделать ход в данный пункт, был ли данный камень поставлен недавно и т.д. Единственное нетривиальное свойство, которое используется, — это угрожает ли данной группе захват в лестницу. Всего используется 48 бинарных свойств (свойства, выраженные целым числом, представляются при помощи унитарного кода). Таким образом каждая позиция представлена в виде таблицы бит 19x19x48.

Стратегическая сеть

Чтобы не учитывать очень плохие ходы и, тем самым, уменьшить степень ветвления при поиске, AlphaGo использует стратегические сети (policy networks) - нейронные сети, которые помогают выбрать хороший ход.

Одна из этих сетей (SL policy networks) может предсказать курс, который профессионал сделает в этой позиции. Эта 13-уровневая нейронная сеть получается путем обучения «с учителем» (supervised learning, SL) на 30 миллионах позиций, взятых из 160 тысяч игр, сыгранных на сервере KGS игроками с 6 по 9 дан. Обучение проходило в течение четырех недель на 50 графических процессорах.

В качестве алгоритма обучения для поиска максимальной вероятности использовался стохастический градиентный спуск. Полученная нейронная сеть рассчитала распределение вероятностей среди всех возможных ходов в данной позиции (представленных, как описано выше). В результате нейронная сеть была в состоянии правильно предсказать курс, выбранный человеком в 57% тестовых ситуаций (не использованных в обучении). Для сравнения, лучший результат до AlphaGo составил 44%. Даже небольшое увеличение точности предсказания значительно увеличивает силу игры.

Стратегическая сеть способна играть самостоятельно, выбирая каждый раз случайный ход с вычисленной вероятностью.

Улучшенная стратегическая сеть

Стратегическая сеть была улучшена с помощью обучения с подкреплением (reinforcement learning, RL), а именно: сеть постоянно улучшалась, играя с одной из сетей, полученных ранее. Более того, каждый раз случайная сеть выбиралась из ранее полученных, чтобы избежать переобучения (ситуации, когда программа выбирает лучший ход, предполагая, что противник использует ту же стратегию, но может плохо сыграть против другого противника). Результатом стала стратегическая сеть (RL policy network), которая превзошла исходную сеть в 80% игр.

Оказалось, что получившаяся стратегическая сеть, без использования функции оценки или перечисления опций, смогла выиграть 85% игр против самой мощной в то время программы Pachi с открытым исходным кодом. Для сравнения, до этого лучшая программа, в которую играли без перечисления опций, а только с помощью сверточной нейронной сети, выиграла 11% игр у Pachi. Таким образом, AlphaGo, не перечисляя опций, смог достичь примерно любителя третьего уровня, указанного авторами программы.

Быстрая стратегическая сеть

Чтобы смоделировать игру методом Монте-Карло, AlphaGo использует более быструю, но менее точную версию стратегической сети (rollout policy), которая получает ответ всего за 2 мкс. Эта быстрая сеть предсказывает ход человека с вероятностью 30%, в то время как улучшенная стратегическая сеть дает ответ на том же оборудовании в течение 3 мс с вероятностью 57%.

Оценочная сеть

Чтобы уменьшить глубину поиска, AlphaGo использовала оценочную сеть. Эта нейронная сеть оценивает вероятность выигрыша в данной позиции. Эта сеть является результатом обучения на 30 миллионах позиций, полученных в результате игры сама с собой в улучшенной стратегической сети. В этом случае в каждой игре было выбрано не более одной позиции (чтобы избежать переобучения из-за сходства позиций в одной игре). Для каждой из этих позиций была оценена вероятность выигрыша по методу Монте-Карло: был организован турнир из множества партий, в котором улучшенная стратегическая сеть, построенная на последнем этапе, играла сама с собой, начиная с этой позиции. После этого сеть оценки прошла обучение по этим данным. Обучение заняло одну неделю на 50 графических процессорах. Результатом стала сеть, которая могла предсказать вероятность выигрыша для каждой позиции, используя при этом в 15 000 раз меньше вычислений, чем метод Монте-Карло.

Дерево поиска решения

AlphaGo перебирает варианты, используя метод Монте-Карло для поиска в дереве, таким образом, что строит частичное игровое дерево из текущей позиции, производя многочисленные игровые симуляции. Для каждого хода в дереве записывается оценка, которая особым образом зависит от оценок ходов, полученных с использованием стратегических и оценочных сетей, от результата случайных игр в предыдущих симуляциях и от количества предыдущих симуляций, которые выбирали этот ход (чем чаще этот ход, тем ниже оценка, чтобы программа учитывала более разнообразные ходы).

В начале каждой симуляции AlphaGo выбирает ход в уже построенном дереве с максимальным счетом. Когда симуляция достигает позиции, которой нет в дереве, эта позиция добавляется в дерево вместе со всеми ходами, разрешенными в этой позиции, которые оцениваются с использованием стратегической сети. Далее, как и в методе Монте-Карло, игра моделируется до конца без ветвления. В этом моделировании каждый ход выбирается случайным образом с вероятностью, полученной с помощью быстрой стратегической сети.

В конце моделирования, в зависимости от результата, оценки ходов в построенном дереве обновляются. Таким образом, каждая симуляция начинается с текущей игровой позиции, доходит до конца, и в результате одной симуляции раскрывается одна позиция в текущем дереве.

Авторы программы обнаружили, что на данном этапе выгоднее использовать не улучшенную стратегическую сеть, а начальную (SL policy network). По мнению авторов, это связано с тем, что профессиональные игроки выбирают более разнообразные ходы, чем улучшенная сеть, что позволяет программе рассмотреть больше вариантов. Таким образом, улучшенная стратегическая сеть не используется во время игры, но ее использование необходимо для построения оценочной сети, когда программа учится, играя сама с собой.

Аппаратные средства

Обучение AlphaGo проходило в течение нескольких недель на 50 графических процессорах с использованием платформы Google Cloud.

AlphaGo был протестирован в компьютерных системах с разным количеством процессоров и графических процессоров, работающих параллельно или распределенными. В каждом случае, на ход было дано 2 секунды.

Версия, выигравшая Fan Hui в октябре 2015 года, работала на 1202 процессорах и 176 графических процессорах.

В игре с Ли Седолем в марте 2016 года AlphaGo использовала 1920 процессоров и 280 графических процессоров, работающих в распределенной сети.

**Анализ:**

**Чем отличается выбранная вами на рассмотрение архитектура нейронной сети от других архитектур?**

Особенностью выбранной архитектуры является то, что в AlphaGo нейросети используются как для оценки позиции (это, фактически, распознавание образа на "картинке" - игровой доске 19х19 с черными и белыми камнями), так и для управления перебором.
Сочетание этих двух факторов дает такую чудовищную силу игры.

**В чем плюсы и минусы данной архитектуры?**

Плюсы:
- огромная сила игры - лучшая в мире;
- архитектура создана на базу хорошо изученных сверточных нейросетей.

Минусы:
- требовательность к ресурсам. Обычный пользователь (любитель Го) не имеет доступа к таким ресурсам.

**Какие могут возникнуть трудности при применении данной архитектуры на практике?**

- высокое энергопотребление;
- необходимость в очень мощных TPU.